In [1]:
from selenium import webdriver 
driver = webdriver.Chrome("/Users/jeonmyeong-un/Desktop/chromedriver")

url = "https://www.melon.com/chart/index.htm"

driver.get(url)

In [2]:
import time
# 차트파인더 클릭
driver.find_element_by_xpath('//*[@id="gnb_menu"]/ul[1]/li[1]/div/div/button/span').click()
time.sleep(1)
# 월간차트 클릭
driver.find_element_by_xpath('//*[@id="d_chart_search"]/div/h4[2]/a').click()
time.sleep(1)
# 연대선택 2020년 클릭
driver.find_element_by_xpath('//*[@id="d_chart_search"]/div/div/div[1]/div[1]/ul/li[1]/span/label').click()
time.sleep(1)
# 연도선택 2021년 클릭
driver.find_element_by_xpath('//*[@id="d_chart_search"]/div/div/div[2]/div[1]/ul/li[1]/span/label').click()
time.sleep(1)
# 월선택 1월 클릭
driver.find_element_by_xpath('//*[@id="d_chart_search"]/div/div/div[3]/div[1]/ul/li[1]/span/label').click()
time.sleep(1)
# 장르선택 종합 클릭
driver.find_element_by_xpath('//*[@id="d_chart_search"]/div/div/div[5]/div[1]/ul/li[1]/span/label').click()
time.sleep(1)
# 검색버튼 클릭
driver.find_element_by_xpath('//*[@id="d_srch_form"]/div[2]/button/span/span').click()

In [18]:
# 월별 음악차트 50개 음악 번호 추출
from bs4 import BeautifulSoup
import urllib.request
import re
# 더보기
a_all = driver.find_elements_by_xpath('//*[@id="lst50"]/td[4]/div/a')
# 제목, 가수, 앨범명, 발매일, 장르
title = []
artists = []
album = []
release_date = []
genre = []
for i in a_all:
    # (1)a.href values
    num = i.get_attribute('href')
    # (1)에서 숫자만 빼오기
    song_num = re.findall(r'\d+', num)
    # list -> string
    song_num = ",".join(song_num)
    
    # URL 결합
    song_url = "https://www.melon.com/song/detail.htm?songId=" + str(song_num)
    # 나의 헤더 찾기 [참조](https://wikidocs.net/66821)
    header =  {"User-Agent":'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36'}
    sequens = urllib.request.Request(song_url, headers = header)
    page = urllib.request.urlopen(sequens)
    soup = BeautifulSoup(page, 'html.parser', from_encoding="CP949")
    
    
    song_info = soup.select('#downloadfrm > div > div')
    dl_list = soup.select('#downloadfrm > div > div > div.entry > div.meta > dl')
    like_list = soup.select("#downloadfrm > div > div > div.entry > div.button.d_song_like")

#   #제목
    for j in song_info:
        name = j.select_one('div.entry > div.info > div.song_name')
    title.append(name.get_text(strip=True).replace("곡명", ""))
    
    #가수
    for art in song_info:
        try:
            artist = art.select_one("div.artist").text.strip()
            artists.append(artist)
        except:
            artist_group = art.select_one("div.atist_view.d_layer.d_dropdown > ul").text.strip()
            for artist in artist_group:
                    artists.append(artist)

    #앨범
    for k in dl_list:
        album_all = k.select_one('#downloadfrm > div > div > div.entry > div.meta > dl > dd:nth-child(2) > a').text
    album.append(album_all)

    #발매일
    for date in dl_list:
        release = date.select_one("#downloadfrm > div > div > div.entry > div.meta > dl > dd:nth-child(4)").text
    release_date.append(release)

    #장르
    for genre_sel in dl_list:
        genre_select = genre_sel.select_one("#downloadfrm > div > div > div.entry > div.meta > dl > dd:nth-child(6)").text
    genre.append(genre_select)

import pandas as pd

data = {
    '제목': title,
    "가수": artists,
    '앨범': album,
    '발매일': release_date,
    '장르': genre }

melon_chart = pd.DataFrame(data)
melon_chart